## **Import Necessary Libraries**

In [ ]:
import numpy as np
from liblinear.liblinearutil import save_model, load_model, train, predict
import matplotlib.pyplot as plt
import h5py

## **Import Functions from train.py**

In [ ]:
from train import get_data, spatial_frequency_feature_fusion, prepare_data, train_linear_model, visualize

## **Run this for preprocessing**

- must include /utils
- example path: "/Users/Danniel/Detection-of-GAN-Generated-Images-using-Spatial-Frequency-Domain-Fusion-Data/utils"

In [ ]:
import sys
# path of the folder "Detection-of-GAN-Generated-Images-using-Spatial-Frequency-Domain-Fusion-Data"
sys.path.append("/Users/Danniel/Detection-of-GAN-Generated-Images-using-Spatial-Frequency-Domain-Fusion-Data/utils")

from preprocessing_save import load_image, save_image

# directory for reading
img_gan = "/Users/Danniel/Downloads/Datasets/Datasets Scenes/Train/GAN"
img_real = "/Users/Danniel/Downloads/Datasets/Datasets Scenes/Train/Real"

# directory for saving (must be an empty folder)
save_dir_gan = "/Users/Danniel/Downloads/Datasets/Datasets Scenes/Train/P_GAN"
save_dir_real = "/Users/Danniel/Downloads/Datasets/Datasets Scenes/Train/P_Real"

# store the images
image_real = load_image(img_real)
image_gan = load_image(img_gan)

save_image(image_real, save_dir_real)
save_image(image_gan, save_dir_gan)

## **Provide directory for Preprocessed Real and GAN-Generated Images**

In [ ]:
gan_directory = "/Users/Danniel/Downloads/Datasets/Datasets Scenes/Train/P_GAN"
real_directory = "/Users/Danniel/Downloads/Datasets/Datasets Scenes/Train/P_Real"

## *"get_data" function will load the preprocessed images from the directory and store it in a list*

In [ ]:
# run data preparation
gan_data = get_data(gan_directory)
real_data = get_data(real_directory)

## *Run to perform the proposed "spatial frequency feature fusion" method to extract meaningful features of an image to classify whether it is GAN-Generated or Real. The feature vector of each image will be stored in "fused_features" list*

In [ ]:
# run feature extraction & feature fusion
extracted_gan = spatial_frequency_feature_fusion(gan_data)
extracted_real = spatial_frequency_feature_fusion(real_data)

## **For Data Visualization**
- provide real and gan parameter

In [ ]:
# visualize
visualize(extracted_gan, extracted_real)

## **Data Preparation before feeding to the classifier**
- assigning of labels
- flattening of feature vector

In [ ]:
# prepare the data
labels, datasets = prepare_data(extracted_gan, extracted_real)

## **Store model data in hdf5 file**

In [ ]:
import os

# provide directory for data folder
os.chdir("/Users/Danniel/Downloads/File")

# rename the model data
hf = h5py.File('data_faces_sample.h5', 'w')

# store labels and datasets to the model data
hf.create_dataset('labels', data=labels)
hf.create_dataset('feature_vector',  data=datasets)

# close the h5py
hf.close()

## **Load hdf5 model data**

In [ ]:
# directory of data folder

import os
os.chdir("/Users/Danniel/Downloads/File")

# provide filename of the model data
rf = h5py.File('data.h5', 'r')

rf.keys()

In [ ]:
loaded_labels = rf.get('labels')
loaded_feature_vector = rf.get('feature_vector')

# convert object to numpy array
np_labels = np.array(loaded_labels)
np_features = np.array(loaded_feature_vector)

print("Loaded Labels: ",len(np_labels))
print("Loaded Features: ", len(np_features))

## **Model Training**
- Liblinear


***
    - change the parameter if the data are from loaded file
    - train_model(np_labels, np_features)

In [ ]:
# train the data
# model = train_linear_model(labels, datasets, C=1)


model = train(np_labels, np_features, f'-s 1 -c 1 -B 1 -w1 10.0 -v 5')

## **Save the model using ".model" extension**

In [ ]:
# save the model
model_file = "/Users/Danniel/Downloads/Model/True Model/scenes.model"
save_model(model_file, model)

## **Load the model**

In [ ]:
model_file = load_model("/Users/Danniel/Downloads/Model/True Model/faces_new.model")

## **Platt Scaling**
- run this to provide the platt scaling of the model

In [ ]:
_, _, scores = predict(labels, datasets, model_file)

validate = train(labels, scores, '-s 0 -c 1 -B 1 -v 5')
platt_scale = train(labels, scores, '-s 0 -c 1 -B 1')

save_model('/Users/Danniel/Downloads/Model/True Platt Scaling/platt_scale_scenes.model', platt_scale)

## **Model Testing**
- provide directory (folder) for testing datasets

## *Testing Combined GAN and Real Images*

In [ ]:
from preprocessing import preprocessing
import os
import numpy as np
from train import spatial_frequency_feature_fusion

# load platt scaler
plat_file = load_model("platt scaler/platt_scale_faces.model")

# test the model
def get_test_data(directory):
    images = []

    # load the images and store in images list
    for filename in os.listdir(directory):
        image = os.path.join(directory, filename)
        if image is not None:
            images.append(image)

    # preprocessing
    preprocessed_img = []
    for i in images:
        preprocessed_img.append(preprocessing(i))   

    # flatten the feature vector
    fused_features = spatial_frequency_feature_fusion(preprocessed_img)
    labels = np.ones((len(fused_features), 1)) 
    true_label = labels.reshape(labels.shape[0])

    feature_vector = []
    for i in fused_features:
        print(i)
        feature_vector.append(i.flatten())


    return feature_vector


def prepare_test_data(gan, real):
    print("\n\n-------------------PREPARING TEST DATA----------------------------\n")
    #label real  and gan datasets
    gan_label = np.ones((len(gan), 1))
    real_label = np.zeros((len(real), 1))

    # combine the labels and datasets
    dataset_labels = np.vstack((gan_label, real_label))
    datasets = np.vstack((gan, real))

    feature_vector = [i.flatten() for i in datasets]
    
    true_label = dataset_labels.reshape(dataset_labels.shape[0])

    return feature_vector, true_label


def predict_labels(feature_vector, true_label, model):  
    # predict the result
    print("\n\n-------------------THE MODEL IS PREDICTING----------------------------\n")
    _, _, svm_scores= predict(true_label, feature_vector, model)
    predicted_labels, _, probability_estimates = predict(labels, svm_scores, plat_file, '-b 1')



    print("------------------------------------------RESULT-----------------------------------\n")
    result = []
    for i in predicted_labels:
        if i == 1.0:
            result.append("GAN")
        elif i == 0.0:
            result.append("Real")

    
    print(result)
    print(probability_estimates)


    return svm_scores, predicted_labels



while True:
    option = input("Does your data came from text file? (y/n)")

    if option.lower() == "y":
        # mode prediction
        scores = predict_labels(np_features, np_labels, model_file)
        break

    elif option.lower() == "n":           
        #provide directory for testing dataset
        dir_gan = "/Users/Danniel/Downloads/Datasets/Datasets Faces/Test/GAN"
        dir_real = "/Users/Danniel/Downloads/Datasets/Datasets Faces/Test/Real"

        # undergo spatial-frequency-feature fusion
        gan = get_test_data(dir_gan)
        real = get_test_data(dir_real)

        # combine the real and gan data
        features, labels = prepare_test_data(gan, real)

        # model prediction
        scores, predicted_labels = predict_labels(features, labels, model_file)
        break
    
    else:
        print("Invalid Input")

In [1]:
import os
import csv

dir = "/Users/Danniel/Downloads/Datasets/Datasets Animals/Test/Real"

filename = os.listdir(dir)

with open('datasets.csv', 'a', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # csv_writer.writerow(['Filename', 'True Labels'])

    for file in filename:
        csv_writer.writerow([file, 0])

In [18]:
import cv2
import os

dir = "/Users/Danniel/Downloads/Pixelated/GAN"

if os.path.isdir(dir):
    pass
else:
    os.makedirs(dir)

def pixelate(image, i):
    # Load the image in grayscale
    image = cv2.imread(image)

    pixelated_image = cv2.resize(image, dsize=(64, 64))
    cv2.imwrite(f"/Users/Danniel/Downloads/Pixelated/GAN/pixelated_gan({i}).png", pixelated_image)

def progress_bar(progress, total):
    percent = 100 * (progress / float(total))
    bar = '▮' * int(percent) + '-' * (100 - int(percent))
    print(f"\r|{bar} | {percent: .2f}%", end="\r")    


# load preprocessed images
directory = "/Users/Danniel/Downloads/combined class test gan"
img = []
for filename in os.listdir(directory):
    image = os.path.join(directory, filename)
    if image is not None:
        img.append(image)   

for index, image in enumerate(img):
    pixelate(image, index)
    progress_bar(index + 1, len(img))